In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

# data

# part 2 80/20

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

le = LabelEncoder()

### tuning model - XG

In [4]:
from xgboost import XGBClassifier

In [12]:
df = pd.read_csv('./data/2022-06/train.csv')
df

,眾間,羅門遠東,氣管,神化,無涉標,死當然,看看分享,闖盪,近季,金臺企銀,...,監察人,益股,自來請益,效益台股,省稅,鋁價,東科華晶科德微中,豐田,label,post_time
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11


In [6]:
X_df = df.iloc[:,0:-2]
y_df = df['label']

# 為了符合xgboost，所有label都加一，讓label=[0,1,2]
y_data = le.fit_transform(y_df)

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X_df.values, y_data, test_size=0.2, random_state=24)

調參code https://blog.csdn.net/weixin_41988628/article/details/83098130   
調參step https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

### Trail 1   
max_depth, min_child_weight

In [8]:
model = XGBClassifier()
# parameters 
param_grid = {
    'max_depth' : range(3,10,2),
    'min_child_weight' : range(1,6,2),
}

Kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state=24)

grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', n_jobs=-1, cv=Kfold)
grid_result = grid_search.fit(X_train, y_train)
print('Best score:', grid_result.best_score_)
print('Best params:', grid_result.best_params_)

# print all trial
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print('%f with:  %r' %(mean, param))

Best score: 0.5112244897959184
Best params: {'max_depth': 7, 'min_child_weight': 1}
0.491837 with:  {'max_depth': 3, 'min_child_weight': 1}
0.475510 with:  {'max_depth': 3, 'min_child_weight': 3}
0.476531 with:  {'max_depth': 3, 'min_child_weight': 5}
0.510204 with:  {'max_depth': 5, 'min_child_weight': 1}
0.493878 with:  {'max_depth': 5, 'min_child_weight': 3}
0.495918 with:  {'max_depth': 5, 'min_child_weight': 5}
0.511224 with:  {'max_depth': 7, 'min_child_weight': 1}
0.491837 with:  {'max_depth': 7, 'min_child_weight': 3}
0.491837 with:  {'max_depth': 7, 'min_child_weight': 5}
0.509184 with:  {'max_depth': 9, 'min_child_weight': 1}
0.489796 with:  {'max_depth': 9, 'min_child_weight': 3}
0.484694 with:  {'max_depth': 9, 'min_child_weight': 5}


### Trail 2   
gamma, learning_rate

In [11]:
model = XGBClassifier()
# parameters 
param_grid = {
    'max_depth' : [7],
    'min_child_weight' : [1],
    'learning_rate' : [0.1, 0.2, 0.3, 0.4, 0.5],
    'gamma' : [0.1, 0],
}

Kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state=24)

grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', n_jobs=-1, cv=Kfold)
grid_result = grid_search.fit(X_train, y_train)
print('Best score:', grid_result.best_score_)
print('Best params:', grid_result.best_params_)

# print all trial
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print('%f with:  %r' %(mean, param))

Best score: 0.5183673469387755
Best params: {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1}
0.491837 with:  {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1}
0.509184 with:  {'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 7, 'min_child_weight': 1}
0.507143 with:  {'gamma': 0.1, 'learning_rate': 0.3, 'max_depth': 7, 'min_child_weight': 1}
0.518367 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1}
0.508163 with:  {'gamma': 0.1, 'learning_rate': 0.5, 'max_depth': 7, 'min_child_weight': 1}
0.502041 with:  {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1}
0.509184 with:  {'gamma': 0, 'learning_rate': 0.2, 'max_depth': 7, 'min_child_weight': 1}
0.511224 with:  {'gamma': 0, 'learning_rate': 0.3, 'max_depth': 7, 'min_child_weight': 1}
0.512245 with:  {'gamma': 0, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1}
0.496939 with:  {'gamma': 0, 'learning_rate': 0.5, 'max_dept

### Trail 3   
subsample, colsample_bytree, reg_alpha

In [12]:
model = XGBClassifier()
# parameters 
param_grid = {
    'max_depth' : [7],
    'min_child_weight' : [1],
    'learning_rate' : [0.4],
    'gamma' : [0.1],
    'subsample' : [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree' : [0.6, 0.7, 0.8, 0.9],
}

Kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state=24)

grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', n_jobs=-1, cv=Kfold)
grid_result = grid_search.fit(X_train, y_train)
print('Best score:', grid_result.best_score_)
print('Best params:', grid_result.best_params_)

# print all trial
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print('%f with:  %r' %(mean, param))

Best score: 0.5010204081632653
Best params: {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.6}
0.501020 with:  {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.6}
0.498980 with:  {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.7}
0.486735 with:  {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.8}
0.492857 with:  {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.9}
0.475510 with:  {'colsample_bytree': 0.7, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.6}
0.497959 with:  {'colsample_bytree': 0.7, 'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.7}
0.486735 with:  {'colsample

worse

In [13]:
model = XGBClassifier()
# parameters 
param_grid = {
    'max_depth' : [7],
    'min_child_weight' : [1],
    'learning_rate' : [0.4],
    'gamma' : [0.1],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
}

Kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state=24)

grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', n_jobs=-1, cv=Kfold)
grid_result = grid_search.fit(X_train, y_train)
print('Best score:', grid_result.best_score_)
print('Best params:', grid_result.best_params_)

# print all trial
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print('%f with:  %r' %(mean, param))

Best score: 0.5193877551020408
Best params: {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 1e-05}
0.518367 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 0.01}
0.508163 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 0.1}
0.497959 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 1}
0.358163 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 100}


### Trail 4  
n_estimators

In [14]:
model = XGBClassifier()
# parameters 
param_grid = {
    'max_depth' : [7],
    'min_child_weight' : [1],
    'learning_rate' : [0.4],
    'gamma' : [0.1],
    'reg_alpha':[1e-5],
    'n_estimators': range(200, 2000, 200),
}

Kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state=24)

grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', n_jobs=-1, cv=Kfold)
grid_result = grid_search.fit(X_train, y_train)
print('Best score:', grid_result.best_score_)
print('Best params:', grid_result.best_params_)

# print all trial
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print('%f with:  %r' %(mean, param))

Best score: 0.5193877551020408
Best params: {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 200, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 200, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 400, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 600, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 800, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 1000, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 1200, 'reg_alpha': 1e-05}
0.519388 with:  {'gamma': 0.1, 'learnin

### final version model   
Best params: {'gamma': 0.1, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 1, 'reg_alpha': 1e-05}
n_estimators = range(200,2000)

In [15]:
xg = XGBClassifier(n_estimators = 200,
                    learning_rate = 0.4,
                    max_depth=7,
                    colsample_bytree=0.86,
                    gamma = 0.1,
                    min_child_weight=1, 
                    reg_alpha=1e-05,
                    seed=24,
                    )

In [17]:
xg.fit(X_train, y_train)
xg.score(X_train, y_train)

0.8275510204081633

In [20]:
test_data = pd.DataFrame()
test_data['label'] = y_test
test_data['predict_label'] = xg.predict(X_test)
test_data

,label,predict_label
0,0,2
1,0,0
2,0,0
3,1,0
4,2,2
...,...,...
240,0,2
241,1,1
242,0,2
243,2,2


In [21]:
print('預測準確率:', accuracy_score(test_data.label, test_data.predict_label))

預測準確率: 0.4897959183673469


### predict all data

In [26]:
df_all = pd.read_csv('yuanta_vector.csv')
df_all

,資產出,總營,滿手金融,深市,鴻智原,沖心得,詮彩晶,益巴菲特,面像,聚愛,...,解惑請,粉紅,趨勢以上,腦正,爆真心,股息宣告,重設,溜滑,label,post_time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-09-03
8588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-09-03
8589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-09-03
8590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2023-11-18


拿掉不出手的資料

In [41]:
xg = XGBClassifier(n_estimators = 200,
                    learning_rate = 0.4,
                    max_depth=7,
                    colsample_bytree=0.86,
                    gamma = 0.1,
                    min_child_weight=1, 
                    reg_alpha=1e-05,
                    seed=24,
                    objective= 'binary:logistic', #僅兩類別
                    )

In [34]:
df = df_all[df_all['label']!=0]

In [13]:
X_df = df.iloc[:,0:-2]
y_df = df['label']

# 為了符合xgboost，label=[-1,1] => [0,1]
le = LabelEncoder()
y_data = le.fit_transform(y_df)

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X_df.values, y_data, test_size=0.2, random_state=24)

In [42]:
xg.fit(X_train, y_train)
xg.score(X_train, y_train)

0.6303366174055829

In [43]:
test_data = pd.DataFrame()
test_data['label'] = y_test
test_data['predict_label'] = xg.predict(X_test)
print('預測準確率:', accuracy_score(test_data.label, test_data.predict_label))

預測準確率: 0.5430680885972108


In [44]:
confusion_matrix(test_data.label, test_data.predict_label)

array([[484, 125],
       [432, 178]], dtype=int64)

# part3

In [6]:
import datetime
from datetime import timedelta

In [7]:
dirPath = './data'
all_data = os.listdir(dirPath) # data所有資料夾

In [8]:
def init_datetime(test_df):
    """
    input: test dataset
    output: test_startDate, test_finalDate, train_startDate, train_endDate
    """
    # 紀錄要預測的日期:要預測的第一天
    ## loop: test_startDate += datetime.timedelta(days=1)
    test_startDate = datetime.datetime.strptime(min(test_df['post_time']), '%Y-%m-%d').date()

    # 預測終止日期
    test_finalDate = datetime.datetime.strptime(max(test_df['post_time']), '%Y-%m-%d').date()

    # 前三十天區間
    train_startDate = test_startDate - datetime.timedelta(days=30)
    train_endDate = test_startDate - datetime.timedelta(days=1)
    
    return test_startDate, test_finalDate, train_startDate, train_endDate

def update_datetime(test_startDate, train_startDate, train_endDate):
    test_startDate = test_startDate + datetime.timedelta(days=1)
    train_startDate = train_startDate + datetime.timedelta(days=1)
    train_endDate = train_endDate + datetime.timedelta(days=1)
    
    return test_startDate, train_startDate, train_endDate

In [13]:
# parameter
threshold = 0.2

# recording result
prediction = pd.DataFrame(columns=['post_time', 'prediction_label', 'true_label'])

In [16]:
%%time
for file in all_data:
    # loading data
    train_filePath = './data/'+file+'/train.csv' #資料夾內 train file
    test_filePath = './data/'+file+'/test.csv' # 資料夾內 test file
    train_df = pd.read_csv(train_filePath)
    test_df = pd.read_csv(test_filePath)
    
    # recording datetime
    test_startDate, test_finalDate, train_startDate, train_endDate = init_datetime(test_df)
    
    # preprocessing data
    ## drop 持平文章(label=0)
    train_df = train_df[train_df['label']!=0]
    test_df = test_df[test_df['label']!=0]
    ## 為了符合xgboost，label=[-1,1] => [0,1]
    train_df['label'] = le.fit_transform(train_df['label'])
    test_df['label'] = le.fit_transform(test_df['label'])
    
    # train model
    X_train = train_df.iloc[:,0:-2]
    y_train = train_df['label']
    xg.fit(X_train, y_train)
    
    # prediction
    while(test_startDate <= test_finalDate):
        # update datetime
        test_date = test_startDate
        test_startDate, train_startDate, train_endDate = update_datetime(test_startDate, train_startDate, train_endDate)

        # df
        X_test = test_df[test_df['post_time'] == str(test_date)].iloc[:,0:-2]
        y_test = test_df[test_df['post_time'] == str(test_date)]['label']

        # avoid no trading day
        if X_train.shape[0] == 0 or X_test.shape[0] == 0:
            # update datetime
            test_startDate, train_startDate, train_endDate = update_datetime(test_startDate, train_startDate, train_endDate)
            continue
        
        print(test_date)
        if (xg.predict(X_test).mean()>0.5) and (abs(xg.predict(X_test).mean()-0.5)> threshold):
            prediction_label = 1
        elif (xg.predict(X_test).mean()<0.5) and (abs(xg.predict(X_test).mean()-0.5)> threshold):
            prediction_label = 0
        else: 
            prediction_label = 'x'

        predict_temp = {'post_time':test_date, 'prediction_label':prediction_label, 'true_label':y_test.mean()}
        prediction.loc[len(prediction)] = predict_temp   

2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-08
2022-06-11
2022-06-18
2022-06-19
2022-06-22
2022-06-23
2022-06-26
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-13
2022-07-14
2022-07-15
2022-07-16
2022-07-21
2022-07-24
2022-07-27
2022-08-03
2022-08-10
2022-08-11
2022-08-12
2022-08-13
2022-08-18
2022-08-21
2022-08-26
2022-08-27
2022-09-01
2022-09-02
2022-09-07
2022-09-08
2022-09-09
2022-09-10
2022-09-11
2022-09-14
2022-09-17
2022-09-18
2022-09-21
2022-09-22
2022-09-23
2022-09-24
2022-10-01
2022-10-02
2022-10-07
2022-10-08
2022-10-09
2022-10-12
2022-10-13
2022-10-14
2022-10-15
2022-10-16
2022-10-19
2022-10-20
2022-10-23
2022-10-26
2022-10-27
2022-10-28
2022-10-29
2022-10-30
2022-11-02
2022-11-03
2022-11-06
2022-11-09
2022-11-10
2022-11-11
2022-11-12
2022-11-13
2022-11-20
2022-11-23
2022-11-24
2022-11-27
2022-11-30
2022-12-01
2022-12-02
2022-12-07
2022-12-08
2022-12-09
2022-12-10
2022-12-11
2022-12-14

In [17]:
prediction

,post_time,prediction_label,true_label
0,2022-06-01,x,1.0
1,2022-06-02,0,0.0
2,2022-06-03,x,0.0
3,2022-06-04,x,1.0
4,2022-06-05,0,0.0
...,...,...,...
203,2024-01-20,1,1.0
204,2024-01-25,1,1.0
205,2024-02-14,1,1.0
206,2024-02-15,1,1.0


In [18]:
no_action = prediction[prediction['prediction_label']=='x']
take_action = prediction[prediction['prediction_label']!='x']
print('預測準確率:', accuracy_score(take_action.true_label.astype(int), take_action.prediction_label.astype(int)))
print('出手率:', len(take_action)/len(prediction))

預測準確率: 0.6194029850746269
出手率: 0.6442307692307693


In [19]:
confusion_matrix(take_action.true_label.astype(int), take_action.prediction_label.astype(int))

array([[49, 19],
       [32, 34]], dtype=int64)